Fonction à importer


In [483]:
# Core libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
# Sklearn functionality
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import export_text
from sklearn import preprocessing
# Keras functionality
import requests
# Ignore warnings
pd.options.mode.chained_assignment = None

Récupération des données

In [484]:
# Import data
stat_equipe_2023 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/Statistiques-Ligue-1#stats_squads_standard_for").content)[0]
stat_equipe_2022 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2021-2022/Stats-2021-2022-Ligue-1#results2021-2022131_overall").content)[0]
stat_equipe_2021 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2020-2021/Stats-2020-2021-Ligue-1#results2020-2021131_overall").content)[0]
stat_equipe_2020 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2019-2020/Stats-2019-2020-Ligue-1#results2019-2020131_overall").content)[0]
stat_equipe_2019 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2018-2019/Stats-2018-2019-Ligue-1#results2018-2019131_overall").content)[0]
matchs_ligue1_2023 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/calendrier/Scores-et-tableaux-Ligue-1#sched_2022-2023_13_1").content)[0]
matchs_ligue1_2022 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2021-2022/calendrier/Calendrier-et-resultats-2021-2022-Ligue-1#sched_all").content)[0]
matchs_ligue1_2021 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2020-2021/calendrier/Calendrier-et-resultats-2020-2021-Ligue-1#sched_all").content)[0]
matchs_ligue1_2020 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2019-2020/calendrier/Calendrier-et-resultats-2019-2020-Ligue-1#sched_2019-2020_13_1").content)[0]
matchs_ligue1_2019 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2018-2019/calendrier/Calendrier-et-resultats-2018-2019-Ligue-1#sched_all").content)[0]

Importer des données pour voir calculer le profit

In [ ]:
df = pd.read_csv("F1.csv")
df =df[["B365H","B365D","B365A","Date","HomeTeam","AwayTeam"]]
df["Date"] =pd.to_datetime(df["Date"])
df.info()

Lier les données

In [485]:
list_concat = [matchs_ligue1_2019,matchs_ligue1_2020, matchs_ligue1_2021, matchs_ligue1_2022, matchs_ligue1_2023]
matchs_ligue1 = pd.concat(list_concat)

Rajouter données utile 2023

In [486]:
matchs_ligue1_2023["but_domicile"] = matchs_ligue1_2023["Score"].str[0]
matchs_ligue1_2023["but_exterieur"] = matchs_ligue1_2023["Score"].str[2]
matchs_ligue1_2023["Points_domicile"] = 0
matchs_ligue1_2023["Points_exterieur"] = 0
matchs_ligue1_2023.loc[matchs_ligue1_2023["but_domicile"]>matchs_ligue1_2023["but_exterieur"],"Points_domicile"] =3 
matchs_ligue1_2023.loc[matchs_ligue1_2023["but_domicile"]<matchs_ligue1_2023["but_exterieur"],"Points_exterieur"] =3
matchs_ligue1_2023.loc[matchs_ligue1_2023["but_domicile"]==matchs_ligue1_2023["but_exterieur"],["Points_domicile","Points_exterieur"]] =1
        

Créer les stats équipe pour chaque journée année 2023

In [487]:
stat_2023=[]
stat_2023.append(stat_equipe_2023[["Équipe", "MJ", "BM", "BE", "Pts"]])
stat_2023[0][['MJ','BM', 'BE', 'Pts',"Domicile"]] = 0
for k in range(37):
    stat_2023.append(stat_2023[k].copy())
    stat_2023[k+1].loc[:, 'MJ'] += 1
    for i in range(len(stat_2023[k+1])):
        donnee_match = matchs_ligue1_2023.loc[(matchs_ligue1_2023["Sem."]==k+1) & ((matchs_ligue1_2023["Domicile"]==stat_2023[k+1].loc[i, 'Équipe'])| (matchs_ligue1_2023["Extérieur"]==stat_2023[k+1].loc[i, 'Équipe']))] 
        if donnee_match.iloc[0,4]==stat_2023[k+1].loc[i, 'Équipe']:
            domicile = 1
        else:
            domicile = 0
        stat_2023[k].loc[i,'Domicile'] = domicile
        stat_2023[k].loc[i,"date"] = donnee_match.iloc[0,2]
        if type(donnee_match.iloc[0,14]) != str:
            continue
        donnee_match.iloc[0,14] = int(donnee_match.iloc[0,14])
        donnee_match.iloc[0,15] = int(donnee_match.iloc[0,15])
        donnee_match.iloc[0,16] = int(donnee_match.iloc[0,16])
        donnee_match.iloc[0,17] = int(donnee_match.iloc[0,17])
        if domicile:
            stat_2023[k+1].loc[i,"Pts"] =(k*stat_2023[k+1].loc[i,"Pts"] + donnee_match.iloc[0,16])/ (k+1)
            stat_2023[k+1].loc[i,"BM"] = (k*stat_2023[k+1].loc[i,"BM"] + donnee_match.iloc[0,14])/ (k+1)
            stat_2023[k+1].loc[i,"BE"] = (k*stat_2023[k+1].loc[i,"BE"] + donnee_match.iloc[0,15])/ (k+1)
        else:
            stat_2023[k+1].loc[i,"Pts"] =(k*stat_2023[k+1].loc[i,"Pts"] + donnee_match.iloc[0,17])/ (k+1)
            stat_2023[k+1].loc[i,"BM"] = (k*stat_2023[k+1].loc[i,"BM"] + donnee_match.iloc[0,15])/ (k+1)
            stat_2023[k+1].loc[i,"BE"] = (k*stat_2023[k+1].loc[i,"BE"] + donnee_match.iloc[0,14])/ (k+1)
    stat_2023[k+1].sort_values(by=['Pts'], ascending=False, inplace=True)
    stat_2023[k+1].reset_index(drop=True, inplace=True)
stats_2023 = pd.concat(stat_2023)


Pareille année 2022

In [488]:
#Clean année 2022
matchs_ligue1_2022 = matchs_ligue1_2022[matchs_ligue1_2022["Tour"] != "Barrage 1/2 Relégation/Promotion France"]
matchs_ligue1_2022.drop(columns="Tour", inplace=True)
matchs_ligue1_2022["but_domicile"] = matchs_ligue1_2022["Score"].str[0]
matchs_ligue1_2022["but_exterieur"] = matchs_ligue1_2022["Score"].str[2]
matchs_ligue1_2022["Points_domicile"] = 0
matchs_ligue1_2022["Points_exterieur"] = 0
matchs_ligue1_2022.loc[matchs_ligue1_2022["but_domicile"]>matchs_ligue1_2022["but_exterieur"],"Points_domicile"] =3 
matchs_ligue1_2022.loc[matchs_ligue1_2022["but_domicile"]<matchs_ligue1_2022["but_exterieur"],"Points_exterieur"] =3
matchs_ligue1_2022.loc[matchs_ligue1_2022["but_domicile"]==matchs_ligue1_2022["but_exterieur"],["Points_domicile","Points_exterieur"]] =1
stat_2022=[]
stat_2022.append(stat_equipe_2022[["Équipe", "MJ", "BM", "BE", "Pts"]])
stat_2022[0][['MJ','BM', 'BE', 'Pts',"Domicile"]] = 0
for k in range(37):
    stat_2022.append(stat_2022[k].copy())
    stat_2022[k+1].loc[:, 'MJ'] += 1
    for i in range(len(stat_2022[k+1])):
        donnee_match = matchs_ligue1_2022.loc[(matchs_ligue1_2022["Sem."]==k+1) & ((matchs_ligue1_2022["Domicile"]==stat_2022[k+1].loc[i, 'Équipe'])| (matchs_ligue1_2022["Extérieur"]==stat_2022[k+1].loc[i, 'Équipe']))] 
        if donnee_match.iloc[0,4]==stat_2022[k+1].loc[i,'Équipe']:
            domicile = 1
        else:
            domicile = 0
        stat_2022[k].loc[i,'Domicile'] = domicile
        stat_2022[k].loc[i,"date"] = donnee_match.iloc[0,2]
        if type(donnee_match.iloc[0,14]) != str:
            continue
        donnee_match.iloc[0,14] = int(donnee_match.iloc[0,14])
        donnee_match.iloc[0,15] = int(donnee_match.iloc[0,15])
        donnee_match.iloc[0,16] = int(donnee_match.iloc[0,16])
        donnee_match.iloc[0,17] = int(donnee_match.iloc[0,17])
        if domicile:
            stat_2022[k+1].loc[i,"Pts"] =(k*stat_2022[k+1].loc[i,"Pts"] + donnee_match.iloc[0,16])/(k+1)
            stat_2022[k+1].loc[i,"BM"] = (k*stat_2022[k+1].loc[i,"BM"] + donnee_match.iloc[0,14])/ (k+1)
            stat_2022[k+1].loc[i,"BE"] = (k*stat_2022[k+1].loc[i,"BE"] + donnee_match.iloc[0,15])/ (k+1)
        else:
            stat_2022[k+1].loc[i,"Pts"] =(k*stat_2022[k+1].loc[i,"Pts"] + donnee_match.iloc[0,17])/(k+1)
            stat_2022[k+1].loc[i,"BM"] = (k*stat_2022[k+1].loc[i,"BM"] + donnee_match.iloc[0,15])/ (k+1)
            stat_2022[k+1].loc[i,"BE"] = (k*stat_2022[k+1].loc[i,"BE"] + donnee_match.iloc[0,14])/ (k+1)
    stat_2022[k+1].sort_values(by=['Pts'], ascending=False, inplace=True)
    stat_2022[k+1].reset_index(drop=True, inplace=True)
stats_2022 = pd.concat(stat_2022)

Année 2021

In [489]:
#Clean année 2021
matchs_ligue1_2021 = matchs_ligue1_2021[matchs_ligue1_2021["Tour"] != "Barrage 1/2 Relégation/Promotion France"]
matchs_ligue1_2021.drop(columns="Tour", inplace=True)
matchs_ligue1_2021["but_domicile"] = matchs_ligue1_2021["Score"].str[0]
matchs_ligue1_2021["but_exterieur"] = matchs_ligue1_2021["Score"].str[2]
matchs_ligue1_2021["Points_domicile"] = 0
matchs_ligue1_2021["Points_exterieur"] = 0
matchs_ligue1_2021.loc[matchs_ligue1_2021["but_domicile"]>matchs_ligue1_2021["but_exterieur"],"Points_domicile"] =3 
matchs_ligue1_2021.loc[matchs_ligue1_2021["but_domicile"]<matchs_ligue1_2021["but_exterieur"],"Points_exterieur"] =3
matchs_ligue1_2021.loc[matchs_ligue1_2021["but_domicile"]==matchs_ligue1_2021["but_exterieur"],["Points_domicile","Points_exterieur"]] =1

stat_2021=[]
stat_2021.append(stat_equipe_2021[["Équipe", "MJ", "BM", "BE", "Pts"]])
stat_2021[0][['MJ','BM', 'BE', 'Pts',"Domicile"]] = 0
for k in range(37):
    stat_2021.append(stat_2021[k].copy())
    stat_2021[k+1].loc[:, 'MJ'] += 1
    for i in range(len(stat_2021[k+1])):
        donnee_match = matchs_ligue1_2021.loc[(matchs_ligue1_2021["Sem."]==k+1) & ((matchs_ligue1_2021["Domicile"]==stat_2021[k+1].loc[i, 'Équipe'])| (matchs_ligue1_2021["Extérieur"]==stat_2021[k+1].loc[i, 'Équipe']))] 
        if donnee_match.iloc[0,4]==stat_2021[k+1].loc[i, 'Équipe']:
            domicile = 1
        else:
            domicile = 0
        stat_2021[k].loc[i,'Domicile'] = domicile
        stat_2021[k].loc[i,"date"] = donnee_match.iloc[0,2]
        if type(donnee_match.iloc[0,14]) != str:
            continue
        donnee_match.iloc[0,14] = int(donnee_match.iloc[0,14])
        donnee_match.iloc[0,15] = int(donnee_match.iloc[0,15])
        donnee_match.iloc[0,16] = int(donnee_match.iloc[0,16])
        donnee_match.iloc[0,17] = int(donnee_match.iloc[0,17])
        if domicile:
            stat_2021[k+1].loc[i,"Pts"] =(k*stat_2021[k+1].loc[i,"Pts"] + donnee_match.iloc[0,16])/(k+1)
            stat_2021[k+1].loc[i,"BM"] = (k*stat_2021[k+1].loc[i,"BM"] + donnee_match.iloc[0,14])/ (k+1)
            stat_2021[k+1].loc[i,"BE"] = (k*stat_2021[k+1].loc[i,"BE"] + donnee_match.iloc[0,15])/ (k+1)
        else:
            stat_2021[k+1].loc[i,"Pts"] =(k*stat_2021[k+1].loc[i,"Pts"] + donnee_match.iloc[0,17])/(k+1)
            stat_2021[k+1].loc[i,"BM"] = (k*stat_2021[k+1].loc[i,"BM"] + donnee_match.iloc[0,15])/ (k+1)
            stat_2021[k+1].loc[i,"BE"] = (k*stat_2021[k+1].loc[i,"BE"] + donnee_match.iloc[0,14])/ (k+1)
        stat_2021[k+1].sort_values(by=['Pts'], ascending=False, inplace=True)
        stat_2021[k+1].reset_index(drop=True, inplace=True)
stats_2021 = pd.concat(stat_2021)

Année 2020

In [490]:
#Clean année 2020
matchs_ligue1_2020["but_domicile"] = matchs_ligue1_2020["Score"].str[0]
matchs_ligue1_2020["but_exterieur"] = matchs_ligue1_2020["Score"].str[2]
matchs_ligue1_2020["Points_domicile"] = 0
matchs_ligue1_2020["Points_exterieur"] = 0
matchs_ligue1_2020.loc[matchs_ligue1_2020["but_domicile"]>matchs_ligue1_2020["but_exterieur"],"Points_domicile"] =3 
matchs_ligue1_2020.loc[matchs_ligue1_2020["but_domicile"]<matchs_ligue1_2020["but_exterieur"],"Points_exterieur"] =3
matchs_ligue1_2020.loc[matchs_ligue1_2020["but_domicile"]==matchs_ligue1_2020["but_exterieur"],["Points_domicile","Points_exterieur"]] =1

stat_2020=[]
stat_2020.append(stat_equipe_2020[["Équipe", "MJ", "BM", "BE", "Pts"]])
stat_2020[0][['MJ','BM', 'BE', 'Pts',"Domicile"]] = 0
for k in range(38):
    stat_2020.append(stat_2020[k].copy())
    stat_2020[k+1].loc[:, 'MJ'] += 1
    for i in range(len(stat_2020[k+1])):
        donnee_match = matchs_ligue1_2020.loc[(matchs_ligue1_2020["Sem."]==k+1) & ((matchs_ligue1_2020["Domicile"]==stat_2020[k+1].loc[i, 'Équipe'])| (matchs_ligue1_2020["Extérieur"]==stat_2020[k+1].loc[i, 'Équipe']))] 
        if donnee_match.iloc[0,4]==stat_2020[k+1].loc[i, 'Équipe']:
            domicile = 1
        else:
            domicile = 0
        stat_2020[k].loc[i,'Domicile'] = domicile
        stat_2020[k].loc[i,"date"] = donnee_match.iloc[0,2]
        if type(donnee_match.iloc[0,14]) != str:
            continue
        donnee_match.iloc[0,14] = int(donnee_match.iloc[0,14])
        donnee_match.iloc[0,15] = int(donnee_match.iloc[0,15])
        donnee_match.iloc[0,16] = int(donnee_match.iloc[0,16])
        donnee_match.iloc[0,17] = int(donnee_match.iloc[0,17])
        if domicile:
            stat_2020[k+1].loc[i,"Pts"] =(k*stat_2020[k+1].loc[i,"Pts"] + donnee_match.iloc[0,16])/(k+1)
            stat_2020[k+1].loc[i,"BM"] = (k*stat_2020[k+1].loc[i,"BM"] + donnee_match.iloc[0,14])/ (k+1)
            stat_2020[k+1].loc[i,"BE"] = (k*stat_2020[k+1].loc[i,"BE"] + donnee_match.iloc[0,15])/ (k+1)
        else:
            stat_2020[k+1].loc[i,"Pts"] =(k*stat_2020[k+1].loc[i,"Pts"] + donnee_match.iloc[0,17])/(k+1)
            stat_2020[k+1].loc[i,"BM"] = (k*stat_2020[k+1].loc[i,"BM"] + donnee_match.iloc[0,15])/ (k+1)
            stat_2020[k+1].loc[i,"BE"] = (k*stat_2020[k+1].loc[i,"BE"] + donnee_match.iloc[0,14])/ (k+1)
        stat_2020[k+1].sort_values(by=['Pts'], ascending=False, inplace=True)
        stat_2020[k+1].reset_index(drop=True, inplace=True)
stats_2020 = pd.concat(stat_2020)

Année 2019

In [491]:
#Clean année 2019
matchs_ligue1_2019 = matchs_ligue1_2019[matchs_ligue1_2019["Tour"] != "Barrage 1/2 Relégation/Promotion France"]
matchs_ligue1_2019.drop(columns="Tour", inplace=True)
matchs_ligue1_2019["but_domicile"] = matchs_ligue1_2019["Score"].str[0]
matchs_ligue1_2019["but_exterieur"] = matchs_ligue1_2019["Score"].str[2]
matchs_ligue1_2019["Points_domicile"] = 0
matchs_ligue1_2019["Points_exterieur"] = 0
matchs_ligue1_2019.loc[matchs_ligue1_2019["but_domicile"]>matchs_ligue1_2019["but_exterieur"],"Points_domicile"] =3 
matchs_ligue1_2019.loc[matchs_ligue1_2019["but_domicile"]<matchs_ligue1_2019["but_exterieur"],"Points_exterieur"] =3
matchs_ligue1_2019.loc[matchs_ligue1_2019["but_domicile"]==matchs_ligue1_2019["but_exterieur"],["Points_domicile","Points_exterieur"]] =1

stat_2019=[]
stat_2019.append(stat_equipe_2019[["Équipe", "MJ", "BM", "BE", "Pts"]])
stat_2019[0][['MJ','BM', 'BE', 'Pts',"Domicile"]] = 0
for k in range(38):
    stat_2019.append(stat_2019[k].copy())
    stat_2019[k+1].loc[:, 'MJ'] += 1
    for i in range(len(stat_2019[k+1])):
        donnee_match = matchs_ligue1_2019.loc[(matchs_ligue1_2019["Sem."]==k+1) & ((matchs_ligue1_2019["Domicile"]==stat_2019[k+1].loc[i, 'Équipe'])| (matchs_ligue1_2019["Extérieur"]==stat_2019[k+1].loc[i, 'Équipe']))] 
        if donnee_match.iloc[0,4]==stat_2019[k+1].loc[i, 'Équipe']:
            domicile = 1
        else:
            domicile = 0
        stat_2019[k].loc[i,'Domicile'] = domicile
        stat_2019[k].loc[i,"date"] = donnee_match.iloc[0,2]
        if type(donnee_match.iloc[0,14]) != str:
            continue
        donnee_match.iloc[0,14] = int(donnee_match.iloc[0,14])
        donnee_match.iloc[0,15] = int(donnee_match.iloc[0,15])
        donnee_match.iloc[0,16] = int(donnee_match.iloc[0,16])
        donnee_match.iloc[0,17] = int(donnee_match.iloc[0,17])
        if domicile:
            stat_2019[k+1].loc[i,"Pts"] =(k*stat_2019[k+1].loc[i,"Pts"] + donnee_match.iloc[0,16])/(k+1)
            stat_2019[k+1].loc[i,"BM"] = (k*stat_2019[k+1].loc[i,"BM"] + donnee_match.iloc[0,14])/ (k+1)
            stat_2019[k+1].loc[i,"BE"] = (k*stat_2019[k+1].loc[i,"BE"] + donnee_match.iloc[0,15])/ (k+1)
        else:
            stat_2019[k+1].loc[i,"Pts"] =(k*stat_2019[k+1].loc[i,"Pts"] + donnee_match.iloc[0,17])/(k+1)
            stat_2019[k+1].loc[i,"BM"] = (k*stat_2019[k+1].loc[i,"BM"] + donnee_match.iloc[0,15])/ (k+1)
            stat_2019[k+1].loc[i,"BE"] = (k*stat_2019[k+1].loc[i,"BE"] + donnee_match.iloc[0,14])/ (k+1)
        stat_2019[k+1].sort_values(by=['Pts'], ascending=False, inplace=True)
        stat_2019[k+1].reset_index(drop=True, inplace=True)
stats_2019 = pd.concat(stat_2019)

Concatener toute les données

In [492]:
stats = pd.concat([stats_2019, stats_2020, stats_2021, stats_2022,stats_2023])
stats["date"]= pd.to_datetime(stats["date"])

Nettoyer les données

In [493]:
# Enlever les colonnes inutiles
matchs_ligue1 = matchs_ligue1[matchs_ligue1["Tour"] != "Barrage 1/2 Relégation/Promotion France"]
matchs_trier= matchs_ligue1[["Date","Domicile","Score","Extérieur","Sem."]]
# Enlever les lignes inutiles
matchs_trier = matchs_trier[matchs_trier["Date"].notna()]
# Mettre les dates au bon format
matchs_trier["Date"] = pd.to_datetime(matchs_trier["Date"])
# Garder les matchs joués
matchs_joue = matchs_trier.dropna(subset=["Score"])
# Remettre les index à 0
matchs_joue.reset_index(drop=True, inplace=True)
# Ajouter les colonnes but_domicile et but_exterieur
matchs_joue["but_domicile"] = matchs_joue["Score"].str[0]
matchs_joue["but_exterieur"] = matchs_joue["Score"].str[2]
# garder les matchs à jouer
matchs_a_jouer = matchs_trier[matchs_trier["Score"].isna()]
# copier les colonnes Domicile et Extérieur dans les parties à jouer pour les retrouver après 
matchs_a_jouer["Domicile2"]=matchs_a_jouer["Domicile"]
matchs_a_jouer["Extérieur2"]=matchs_a_jouer["Extérieur"]
# Mettres les buts en type int
matchs_joue["but_domicile"] = matchs_joue["but_domicile"].astype(int)
matchs_joue["but_exterieur"] = matchs_joue["but_exterieur"].astype(int)
# Ajouter les colonnes nombre de but et écarts de but
matchs_joue["nombre_de_but"] = matchs_joue["but_domicile"] + matchs_joue["but_exterieur"]
matchs_joue["ecart_de_but"] = matchs_joue["but_domicile"] - matchs_joue["but_exterieur"]

# Ajouter les colonnes resultat_domicile et resultat_exterieur
    # matchs_joue["resultat_domicile"] = 0
    # matchs_joue["resultat_exterieur"] = 0
    # for i in range(len(matchs_joue)):
    #     if matchs_joue["but_domicile"][i] > matchs_joue["but_exterieur"][i]:
    #         matchs_joue["resultat_domicile"][i] = 2
    #         matchs_joue["resultat_exterieur"][i] = 0
    #     elif matchs_joue["but_domicile"][i] < matchs_joue["but_exterieur"][i]:
    #         matchs_joue["resultat_domicile"][i] = 0
    #         matchs_joue["resultat_exterieur"][i] = 2
    #     else:
    #         matchs_joue["resultat_domicile"][i] = 1
    #         matchs_joue["resultat_exterieur"][i] = 1
# Diviser la date en jour, mois et année
matchs_joue["jour"] = matchs_joue["Date"].dt.day
matchs_joue["mois"] = matchs_joue["Date"].dt.month
matchs_joue["annee"] = matchs_joue["Date"].dt.year
# Diviser la date en jour, mois et année pour les matchs à jouer
matchs_a_jouer["jour"] = matchs_a_jouer["Date"].dt.day
matchs_a_jouer["mois"] = matchs_a_jouer["Date"].dt.month
matchs_a_jouer["annee"] = matchs_a_jouer["Date"].dt.year
# Enlever les matchs covid
matchs_a_jouer = matchs_a_jouer.loc[matchs_a_jouer["annee"] == 2023]


Lier les données

In [520]:
matchs_joue_dom = pd.merge(matchs_joue, stats,  how='left', left_on=["Domicile","Date"], right_on = ["Équipe","date"])
matchs_joue_dom_ext = pd.merge(matchs_joue_dom, stats,  how='left', left_on=["Extérieur","Date"], right_on = ["Équipe","date",])
matchs_joue_dom_ext.isna().sum()

Date               0
Domicile_x         0
Score              0
Extérieur          0
Sem.               2
but_domicile       0
but_exterieur      0
nombre_de_but      0
ecart_de_but       0
jour               0
mois               0
annee              0
Équipe_x         209
MJ_x             209
BM_x             209
BE_x             209
Pts_x            209
Domicile_y       209
date_x           209
Équipe_y         207
MJ_y             207
BM_y             207
BE_y             207
Pts_y            207
Domicile         207
date_y           207
dtype: int64

In [521]:
matchs_joue_dom_ext = matchs_joue_dom_ext[['Sem.', 'but_domicile',"but_exterieur","jour","mois","annee","Pts_x","BM_x","BE_x","Pts_y","BM_y","BE_y","nombre_de_but","ecart_de_but"]]
matchs_joue_dom_ext.rename(columns={"Pts_x": "Pts_dom", "BM_x": "BM_dom", "BE_x": "BE_dom","Pts_y": "Pts_ext", "BM_y": "BM_ext", "BE_y": "BE_ext"}, inplace=True)

Créer les données de test et d'entrainement

In [525]:
matchs_joue_dom_ext = matchs_joue_dom_ext.dropna(axis=0)
matchs_joue_dom_ext.isna().sum()

Sem.             0
but_domicile     0
but_exterieur    0
jour             0
mois             0
annee            0
Pts_dom          0
BM_dom           0
BE_dom           0
Pts_ext          0
BM_ext           0
BE_ext           0
nombre_de_but    0
ecart_de_but     0
dtype: int64

In [536]:
X = matchs_joue_dom_ext[['Sem.',"Pts_dom","BM_dom","BE_dom","Pts_ext","BM_ext","BE_ext"]]
y = matchs_joue_dom_ext[["nombre_de_but","ecart_de_but"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Encoder les colonnes Domicile et Extérieur pour les mettres en entier


In [506]:
# cols = ['Domicile', 'Extérieur']
# enc = preprocessing.LabelEncoder()
# for col in cols:
#     X_train[col] = X_train[col].astype('str')
#     X_test[col] = X_test[col].astype('str')
#     X_train[col] = enc.fit_transform(X_train[col])
#     X_test[col] = enc.transform(X_test[col])
#     matchs_a_jouer[col] = enc.fit_transform(matchs_a_jouer[col])

Création du modèle décision tree pour le nombre de but

In [537]:
# Build a logistic regression model
mode1_dt = DecisionTreeClassifier()
mode1_dt.fit(X_train, y_train["nombre_de_but"])
# Afficher l'arbre de décision
print("décision_tree pour écart de but \n",export_text(mode1_dt, feature_names=list(X_train)))
# Check the model performance with the training data
predictions1_dt_train = mode1_dt.predict(X_train)
print("prediction avec les training data:",accuracy_score(y_train["nombre_de_but"], predictions1_dt_train))
# Check the model performance with the test data
predictions1_dt_test = mode1_dt.predict(X_test)
print("prediction avec les test data:",accuracy_score(y_test["nombre_de_but"], predictions1_dt_test))
# matchs_a_jouer["nombre_de_but"] = mode1_dt.predict(matchs_a_jouer[["Domicile","Extérieur","jour","mois","annee"]])

décision_tree pour écart de but 
 |--- Pts_dom <= 1.28
|   |--- BM_dom <= 1.49
|   |   |--- BE_dom <= 0.98
|   |   |   |--- Pts_ext <= 0.42
|   |   |   |   |--- Pts_dom <= 0.87
|   |   |   |   |   |--- BE_dom <= 0.60
|   |   |   |   |   |   |--- BM_ext <= 0.33
|   |   |   |   |   |   |   |--- Sem. <= 1.50
|   |   |   |   |   |   |   |   |--- class: 2
|   |   |   |   |   |   |   |--- Sem. >  1.50
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- BM_ext >  0.33
|   |   |   |   |   |   |   |--- class: 5
|   |   |   |   |   |--- BE_dom >  0.60
|   |   |   |   |   |   |--- class: 3
|   |   |   |   |--- Pts_dom >  0.87
|   |   |   |   |   |--- class: 6
|   |   |   |--- Pts_ext >  0.42
|   |   |   |   |--- Pts_dom <= 0.86
|   |   |   |   |   |--- BE_ext <= 0.67
|   |   |   |   |   |   |--- class: 2
|   |   |   |   |   |--- BE_ext >  0.67
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- Pts_dom >  0.86
|   |   |   |   |   |--- BM_dom <= 1.04
|   |   |   |   |

Création du model decision tree pour l'écart de but

In [538]:
# Build a decision tree model
mode1_dt = DecisionTreeClassifier()
mode1_dt.fit(X_train, y_train["ecart_de_but"])
# Afficher l'arbre de décision
print("décision_tree pour écart de but \n",export_text(mode1_dt, feature_names=list(X_train)))
# Check the model performance with the training data
predictions1_dt_train = mode1_dt.predict(X_train)
print("prediction avec les training data:",accuracy_score(y_train["ecart_de_but"], predictions1_dt_train))
# Check the model performance with the test data
predictions1_dt_test = mode1_dt.predict(X_test)
print("prediction avec les test data:",accuracy_score(y_test["ecart_de_but"], predictions1_dt_test))
# matchs_a_jouer["ecart_de_but"] = mode1_dt.predict(matchs_a_jouer[["Domicile","Extérieur","jour","mois","annee"]])

décision_tree pour écart de but 
 |--- Pts_dom <= 1.12
|   |--- BM_ext <= 1.39
|   |   |--- Pts_dom <= 0.61
|   |   |   |--- Pts_dom <= 0.60
|   |   |   |   |--- BM_ext <= 1.23
|   |   |   |   |   |--- BE_dom <= 2.36
|   |   |   |   |   |   |--- BE_dom <= 2.15
|   |   |   |   |   |   |   |--- Pts_ext <= 1.03
|   |   |   |   |   |   |   |   |--- Pts_ext <= 0.96
|   |   |   |   |   |   |   |   |   |--- BM_ext <= 0.91
|   |   |   |   |   |   |   |   |   |   |--- BE_dom <= 1.95
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 6
|   |   |   |   |   |   |   |   |   |   |--- BE_dom >  1.95
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 5
|   |   |   |   |   |   |   |   |   |--- BM_ext >  0.91
|   |   |   |   |   |   |   |   |   |   |--- Sem. <= 31.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 2
|   |   |   |   |   |   |   |   |   |   |--- Sem. >  31.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncat

Prévision des résultats

In [531]:
matchs_a_jouer["Score"] = ((matchs_a_jouer["nombre_de_but"]+matchs_a_jouer["ecart_de_but"])/2).astype(int).astype(str) + " - " + ((matchs_a_jouer["nombre_de_but"]-matchs_a_jouer["ecart_de_but"])/2).astype(int).astype(str)
matchs_prevu = matchs_a_jouer[["Date","Domicile2","Extérieur2","Score","nombre_de_but","ecart_de_but"]]
matchs_prevu.head(15)

,Date,Domicile2,Extérieur2,Score,nombre_de_but,ecart_de_but
287,2023-03-10,Lille,Lyon,1 - 1,3,0
288,2023-03-11,Auxerre,Rennes,1 - 1,3,0
289,2023-03-11,Brest,Paris S-G,1 - 1,3,0
290,2023-03-12,Clermont Foot,Lens,1 - 1,3,0
291,2023-03-12,Lorient,Troyes,1 - 1,3,0
292,2023-03-12,Ajaccio,Montpellier,1 - 1,3,0
293,2023-03-12,Nantes,Nice,1 - 1,3,0
294,2023-03-12,Angers,Toulouse,1 - 1,3,0
295,2023-03-12,Monaco,Reims,1 - 1,3,0
296,2023-03-12,Marseille,Strasbourg,1 - 1,3,0


Modele logisitic regression ecart de but

In [533]:
# Build a logistic regression model
mode1_lr = LogisticRegression(solver = 'liblinear', max_iter=1000, multi_class='ovr')
mode1_lr.fit(X_train, y_train["nombre_de_but"])
# Check the model performance with the training data
predictions1_lr_train = mode1_lr.predict(X_train)
print("prediction avec les training data:",accuracy_score(y_train["nombre_de_but"], predictions1_lr_train))
# Check the model performance with the test data
predictions1_lr_test = mode1_lr.predict(X_test)
print("prediction avec les test data:",accuracy_score(y_test["nombre_de_but"], predictions1_lr_test))
# matchs_a_jouer["nombre_de_but"] = mode1_lr.predict(matchs_a_jouer[["Domicile","Extérieur","jour","mois","annee"]])


prediction avec les training data: 0.2627118644067797
prediction avec les test data: 0.22469135802469137


Modele logistic regression nombre de but

In [535]:
# Buill a logistic regression model for the ecart de but
mode2_lr = LogisticRegression(solver = 'liblinear', max_iter=1000, multi_class='ovr')
mode2_lr.fit(X_train, y_train["ecart_de_but"])
# Check the model performance with the training data
predictions2_lr_train = mode2_lr.predict(X_train)
print("prediction avec les training data:",accuracy_score(y_train["ecart_de_but"], predictions2_lr_train))
# Check the model performance with the test data
predictions2_lr_test = mode2_lr.predict(X_test)
print("prediction avec les test data:",accuracy_score(y_test["ecart_de_but"], predictions2_lr_test))
# matchs_a_jouer["ecart_de_but"] = mode2_lr.predict(matchs_a_jouer[["Domicile","Extérieur","jour","mois","annee"]])


prediction avec les training data: 0.2870762711864407
prediction avec les test data: 0.2740740740740741


Prévision des résultat logistic regression


In [503]:
matchs_a_jouer["Score"] = ((matchs_a_jouer["nombre_de_but"]+matchs_a_jouer["ecart_de_but"])/2).astype(int).astype(str) + " - " + ((matchs_a_jouer["nombre_de_but"]-matchs_a_jouer["ecart_de_but"])/2).astype(int).astype(str)
matchs_prevu = matchs_a_jouer[["Date","Domicile2","Extérieur2","Score","nombre_de_but","ecart_de_but"]]
matchs_prevu.head(15)

,Date,Domicile2,Extérieur2,Score,nombre_de_but,ecart_de_but
287,2023-03-10,Lille,Lyon,1 - 1,3,0
288,2023-03-11,Auxerre,Rennes,1 - 1,3,0
289,2023-03-11,Brest,Paris S-G,1 - 1,3,0
290,2023-03-12,Clermont Foot,Lens,1 - 1,3,0
291,2023-03-12,Lorient,Troyes,1 - 1,3,0
292,2023-03-12,Ajaccio,Montpellier,1 - 1,3,0
293,2023-03-12,Nantes,Nice,1 - 1,3,0
294,2023-03-12,Angers,Toulouse,1 - 1,3,0
295,2023-03-12,Monaco,Reims,1 - 1,3,0
296,2023-03-12,Marseille,Strasbourg,1 - 1,3,0
